# Detecting EMA cross traps by using NEAT

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import os

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import neat

### Load Price Data

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir.parent) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(csv_file)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [4]:
data = dataset.copy()

In [5]:
data = data[data.index > '2020-11-01 00:00:00']

In [6]:
data["ema_fast"] = ta.ema(data["Close"], length=20)
data["ema_low"] = ta.ema(data["Close"], length=250)
data["ema_cross"] = ((data["ema_fast"] > data["ema_low"]) & (data["ema_fast"].shift(1) <= data["ema_low"].shift(1)) | (data["ema_fast"] < data["ema_low"]) & (data["ema_fast"].shift(1) >= data["ema_low"].shift(1)))

## Calculate some common features

In [7]:
data["ATR"] = ta.atr(data["High"], data["Low"], data["Close"], length=14)  # Volatility
data["RSI"] = ta.rsi(data["Close"], length=14)  # Momentum indicator

## TRAP labeling

In [8]:
def is_trap(r):
    trap = ''
    if r['ema_cross'] == True:
        if r['ema_fast'] > r['ema_low']:
            # Cross up
            if r['min_low_1dlater'] < r['Close'] - 3.5:
                trap = 1
            else:
                trap = 0
        else:
            # Cross down
            if r['max_high_1dlater'] > r['Close'] + 3.5:
                trap = 1
            else:
                trap = 0
    return trap

In [9]:
data['max_high_1dlater'] = data['High'].shift(-51).rolling(51).max()
data['min_low_1dlater'] = data['Low'].shift(-51).rolling(51).min()
data['trap'] = data.apply(lambda r: is_trap(r), axis=1)

In [10]:
# cross_data = data[data.ema_cross == True]
# len(cross_data[cross_data.trap == 0]) / len(cross_data['trap'])

## Features

In [11]:
data['ema_fast_s1'] = data['ema_fast'].shift(1)
data['ema_fast_s2'] = data['ema_fast'].shift(2)
data['ema_fast_s3'] = data['ema_fast'].shift(3)
data['ema_fast_s4'] = data['ema_fast'].shift(4)
data['ema_fast_s5'] = data['ema_fast'].shift(5)
data['ema_fast_s6'] = data['ema_fast'].shift(6)
data['ema_fast_s7'] = data['ema_fast'].shift(7)
data['ema_fast_s8'] = data['ema_fast'].shift(8)
cross_up_data = data[(data.ema_cross == True) & (data.ema_fast > data.ema_low)]
cross_up_data.dropna(inplace=True)

In [12]:
len(cross_up_data)

181

In [13]:
X = cross_up_data[['ema_fast', 'ema_fast_s1', 'ema_fast_s2', 'ema_fast_s3', 'ema_fast_s4', 'ema_fast_s5', 'ema_fast_s6', 'ema_fast_s7', 'ema_fast_s8', "trap"]]

# Train-Test Split
X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)

In [14]:
len(X_train)

135

In [17]:
len(X_train[X_train.trap == 0]) / len(X_train[X_train.trap == 1])

0.5340909090909091

In [18]:
X_none_trap = X_train[X_train.trap == 0]
balanced_X_train = pd.concat([X_train, X_none_trap], ignore_index=True)
len(balanced_X_train[balanced_X_train.trap == 0]) / len(balanced_X_train[balanced_X_train.trap == 1])

1.0681818181818181

In [15]:
len(X_test)

46

In [19]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for move_index, row in balanced_X_train.iterrows():
            inputs = [row['ema_fast'], row['ema_fast_s1'], row['ema_fast_s2'], row['ema_fast_s3'], row['ema_fast_s4'], row["ema_fast_s5"], row["ema_fast_s6"], row["ema_fast_s7"], row["ema_fast_s8"]]
            expected_output = row['trap']
            output = net.activate(inputs)
            genome.fitness -= (output[0] - expected_output) ** 2


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    # p.add_reporter(neat.StdOutReporter(True))
    # stats = neat.StatisticsReporter()
    # p.add_reporter(stats)

    # Run for up to 100 generations.
    winner = p.run(eval_genomes, 100)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    return neat.nn.FeedForwardNetwork.create(winner, config)

In [20]:
%%time
config_path = os.path.join(current_dir, 'style-mix-1.cfg')
best_brain = run(config_path)


Best genome:
Key: 14074
Fitness: -41.45054973524408
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.013806589138558756, response=1.0, activation=sigmoid, aggregation=sum)
	385 DefaultNodeGene(key=385, bias=-0.5690971934924585, response=1.0, activation=sigmoid, aggregation=sum)
	476 DefaultNodeGene(key=476, bias=1.1009492472463842, response=1.0, activation=sigmoid, aggregation=sum)
	510 DefaultNodeGene(key=510, bias=-0.5268584501949927, response=1.0, activation=sigmoid, aggregation=sum)
	687 DefaultNodeGene(key=687, bias=-1.1511683284114793, response=1.0, activation=sigmoid, aggregation=sum)
	907 DefaultNodeGene(key=907, bias=0.6389167437354188, response=1.0, activation=sigmoid, aggregation=sum)
	2063 DefaultNodeGene(key=2063, bias=1.3879245707743166, response=1.0, activation=sigmoid, aggregation=sum)
	2151 DefaultNodeGene(key=2151, bias=1.1935436574849456, response=1.0, activation=sigmoid, aggregation=sum)
	2189 DefaultNodeGene(key=2189, bias=-0.19588273151654986, response=1.0, activation=si

In [21]:
best_brain

In [22]:
# Show output of the most fit genome against training data.
outputs = []
for i, row in X_test.iterrows():
    inputs = [row['ema_fast'], row['ema_fast_s1'], row['ema_fast_s2'], row['ema_fast_s3'], row['ema_fast_s4'], row["ema_fast_s5"], row["ema_fast_s6"], row["ema_fast_s7"], row["ema_fast_s8"]]
    expected_output = row['trap']
    output = best_brain.activate(inputs)
    outputs.append(round(output[0]))
    # print("input {!r}, expected output {!r}, got {!r}".format(inputs, expected_output, output))

In [23]:
expected_outputs = X_test['trap'].to_list()
# Evaluate Performance
print("Accuracy:", accuracy_score(expected_outputs, outputs))

Accuracy: 0.2608695652173913
